# <center>City of Calgary Property Assessment Exploratory Data Analysis</center>

### <center>Greg Cameron, Atlanta Liu, Dany Hachem</center>

### Introduction

&nbsp;&nbsp; The City of Calgary releases property assessment values on an annual basis. These assessments play an important role in determining the amount of property taxes that homeowners must pay to maintain city services. The city claims that it does its best to promote fairness and equity in its assessments by utilizing a sales comparison approach (Property Assessment 2019). This approach considers physical characteristics of the residential property, nearby community services, and surrounding property within the community. However, there is comparatively little mention of how external factors could influence the values of a property assessment. For this project, factors of interest that will be examined in relation to property assessment values include economic downturn (oil price), population growth, as well as crime rates. 
   
&nbsp;&nbsp; Our report aims to further our understanding of how accurate these assessments are by looking at the extent to which each factor of interest can affect property assessments. For individuals who do not currently own any residential property, the insight gathered from our analysis should hopefully provide them with a better awareness of how important these factors are when determining whether one should purchase certain residential properties. In general, this analysis will provide a great starting point to discuss why the observed values of residential property assessments are seen to increase and decrease over the years.

&nbsp;&nbsp; The datasets for each factor will be obtained from Open Calgary. Using the data wrangling and visualization methods learned in class, we will be conducting exploratory data analysis of these factors on residential property assessments in Calgary to provide some insight into our guiding questions. All the datasets used are in comma separated value (csv) format and are up to date as of September 2019.

### Guiding Questions

&nbsp;&nbsp; To address how our three factors (economic downturn, population growth, and crime rates) have impacted property assessments, we developed three guiding questions that will be answered through the exploration of our data.

*1.  	How has the economic downturn (decrease in oil price) affected the residential property assessments in Calgary?*

>•	Calgary has been impacted by the cyclical nature of the oil and gas industry since the first oil was discovered in Alberta in the early 20th century.  As Calgary became the centre of Canada’s petroleum industry, the cities’ fortunes have become intrinsically tied to the price of oil. The current downturn has spanned the last 5 years, with the North American benchmark price, West Texas Intermediate (WTI) falling from 112 dollar/barrel in June 2014 to 26 dollar/barrel in 2016.  The recovery has been slow, dramatically impacting the oil company revenues, and ultimately leading to large rounds of lay-offs of oil and gas employees. How has the economic downturn in Calgary impacted property the property values in the city? Has the impact been felt equally throughout the city?

*2.  	How does population growth affect a communities’ property assessments over the years?*

>•	The Calgary Census (2016) reported that Calgary is one of the fastest growing population in Alberta, reaching a population growth rate of 14.6% from 2011 to 2016. Additionally, it has one of the highest home ownership rates compared to all major Canadian cities. Despite the prominence of population change, there has been relatively few articles published regarding the relationship between population growth and property assessment. For the few that do (Weber & Buchanan 1980; Buchanan & Weber 1982), studies have shown an increase in average value of single-family residences with increases in population. While these effects are presumed to be direct (higher demand = high values), the authors have noted that there are also important indirect effects of population change to consider. Potential determinants include examining a community’s population density, average housing age, addition of newer houses, average income, and increased load on local communal services.  
•	While our model will not be looking into as much depth on the direct and indirect effects, this analysis will entail a comparison of several communities that observed the least overall population growth and several communities that observed the highest overall population growth with respect to median property assessment for the years 2005 and 2017. 

*3.  	What is the relationship between crime rates and property assessments?*

>•	All types of crime lead to indirect costs to society at large, which might lead to economic effects further down the line. Although it is difficult to directly attribute declining property assessments to criminal incidents, several studies have shown that there is indeed a link between certain types of crime and a decrease is housing values (Maximino 2017; Campbell 2007). For example, the socio-economic effects of crime in a community is quite visible. New residents may tend to avoid neighborhoods with elevated crime rates, while current residents might look to move elsewhere. On the other hand, neighborhoods that are considered safe might be more expensive due to higher demand, seeing how safety is ranked higher than price when it comes to buying a new home. Visualizing the data on property assessment values and crime rates in select communities in Calgary will further shed some light on whether these two things are related. 


In [2]:
import pandas as pd
import numpy as np

import re

import matplotlib as mpl
import matplotlib.pyplot as plt
import plotly as plotly
import plotly.offline as py
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

#%matplotlib inline
mpl.rcParams['figure.dpi']= 96
mpl.style.use('ggplot')

import geopandas as gpd
import shapely.geometry as sg

import ipywidgets as widgets
from ipywidgets import interact, interact_manual

py.init_notebook_mode(connected=True)

import warnings
warnings.simplefilter(action='ignore')

In [3]:
# get rid of all decimal places, bring them back when needed
pd.options.display.float_format = '{:.0f}'.format

# Main Datasets 

### Property Assessments Dataset

&nbsp;&nbsp; This dataset is the central piece to our investigation. Retrieving it from Open Calgary, the database lists all the property assessment values throughout the entire city between the years 2005-2019, giving us over 6.7 million rows to work with. The overall process of cleaning and filtering involves aggregating all the individual properties that had a complete set of assessment values across the 15 years, and removing any non-residential communities from the set. Finally, the columns are grouped according to community name and year to feature the median property assessment.

&nbsp;&nbsp; An important note is that we made a decision here to omit any properties that were built later on. Properties that were built later in the year could disproportionately affect the overall property assessments in the community of interest and mask the effects of potential trends when analyzing it together with other factors. Additionally, we decided to analyze the median over the mean property assessments, due to the fact that median values are less likely to be subjected to the effects of large outliers. We wanted to avoid having a few large-valued assessments that greatly skewed our analysis.

In [4]:
propassess = pd.read_csv("Property_Assessments.csv")

#### Cleaning & Filtering: Individual Property Assessments 2005-2019

In [5]:
# create 2 dataframes with fewer columns - will merge later
temp1 = propassess[['ROLL_NUMBER', 'ROLL_YEAR', 'ASSESSED_VALUE']]
temp2 = propassess[['ROLL_NUMBER','LATITUDE', 'LONGITUDE', 'COMM_NAME', 'ASSESSMENT_CLASS']].set_index(['ROLL_NUMBER']).groupby('ROLL_NUMBER').first()

# restack temp1 so that all yearly assessments for property are in single row
temp1stack = temp1.set_index(['ROLL_NUMBER', 'ROLL_YEAR']).stack().unstack(1)

# remove second part of index ("ASSESSED_VALUE" was tagged onto every row index)
temp1stack2 = temp1stack.reset_index(level=1)

# merge temp2 (latitude and longitude) into the restacked dataset
merged = temp1stack2.merge(temp2, left_index = True, right_index = True)

In [6]:
# remove any row that contains a NaN (output will only have a property that has value for each year)
PropAssessYear = merged.dropna()
display(PropAssessYear.head())

,level_1,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,LATITUDE,LONGITUDE,COMM_NAME,ASSESSMENT_CLASS
ROLL_NUMBER,,,,,,,,,,,,,,,,,,,,
2004505,ASSESSED_VALUE,145000,149000,203500,246000,335500,299500,281000,365000,363500,397500,285000,285000,295000,304000,963500,51,-114,CALGARY INTERNATIONAL AIRPORT,NR
3000502,ASSESSED_VALUE,78500,81000,96500,99500,123000,111000,120000,110500,110500,124000,112500,116000,121500,126000,130500,51,-114,SADDLE RIDGE INDUSTRIAL,NR
3001401,ASSESSED_VALUE,277000,284500,403000,455500,581000,546500,457000,454000,451500,491500,457500,456500,470000,481000,494000,51,-114,SADDLE RIDGE,RE
3001500,ASSESSED_VALUE,307500,315500,433000,485500,616000,589500,484500,481500,480000,520000,485000,484000,498500,511000,527500,51,-114,SADDLE RIDGE,RE
3001609,ASSESSED_VALUE,275000,282000,397500,456000,578000,544000,460500,459000,458000,499000,465000,465500,481000,494000,509000,51,-114,SADDLE RIDGE,RE


#### Cleaning & Filtering: Property Assessments By Community 2005-2019

In [7]:
#Begin by aggregating all the properties and their assessed values
#Retain only properties that are recorded for all of the 15 years
temp4 = propassess.groupby('ROLL_NUMBER')['ASSESSED_VALUE'].agg('count')==15
temp5 = temp4[temp4==True].to_frame()

#Use a boolean index to match the roll_numbers with all 15 assessed values for the years
#Then locate the roll_numbers for classes that are only residential (excluding: Farm-land & Industrial communities)
propassessselect = propassess[propassess['ROLL_NUMBER'].isin(temp5.index.values.tolist())]
propassessselect2 = propassessselect[propassessselect['ASSESSMENT_CLASS'] == 'RE']

#Finalize dataset by grouping according to the columns we are interested in, and giving them a proper name.
tempAssess1 = propassessselect2[['ROLL_YEAR', 'ASSESSED_VALUE','COMM_NAME']]
tempAssess1.rename(columns={'COMM_NAME': 'Community Name', 'ROLL_YEAR':'Year'}, inplace = True)

#Finalize the dataframe by grouping it according to community name and year, as well as feature the median assessed value.
tempAssess2 = tempAssess1.groupby(['Community Name','Year'])
PropAssessCommYear = tempAssess2.median().dropna()

In [8]:
 display(PropAssessCommYear)

ASSESSED_VALUE
Community Name Year                
01B            2005           76000
               2006          165000
               2007          298500
               2008          395000
               2010          585500
...                             ...
WOODLANDS      2015          467500
               2016          460000
               2017          442500
               2018          452500
               2019          443500

[3439 rows x 1 columns]

### Population Dataset

&nbsp;&nbsp; The Historical Community Population dataset (retrieved from OpenCalgary) features the population count for each community between the years 1968 to 2017. The authors of the dataset noted that the total counts did not include 'residual sub areas', so results will slightly differ from the data published in civic census records. We decided to use this dataset over another dataset ("Calgary's Population 1958-2019") because the 'Historical Community Population' specified community names instead of sectors (ie. North, Southeast...), which was more relevant to our methods of cross-comparison when merging with our crime rates dataset.

&nbsp;&nbsp; To clean this dataset, we began by extracting only the year from a datetime format and restructuring the columns to only include a the community name, year, as well as population count. The column 'census_year' is subsequently dropped since we have extracted the year from its datetime format. Columns are renamed to maintain consistency.

In [9]:
PopTot = pd.read_csv("Historical_Community_Populations.csv")

In [10]:
display(PopTot.head())

,name,comm_code,census_year,population,occupied dwellings,persons per unit,notes
0,HUNTINGTON HILLS,HUN,04/01/1996 12:00:00 AM,14595,5100,3,NaN
1,HARVEST HILLS,HAR,04/01/1969 12:00:00 AM,0,0,0,NaN
2,ROYAL OAK,ROY,04/01/2009 12:00:00 AM,10423,3588,3,NaN
3,MAHOGANY,MAH,04/01/2000 12:00:00 AM,0,0,0,NaN
4,FOREST HEIGHTS,FHT,04/01/1990 12:00:00 AM,6854,2005,3,NaN


#### Cleaning & Filtering: Population By Community 1968-2017

In [11]:
# Converting column 'census-year' to datetime format in order to extract the year
PopTot['census_year'] = pd.to_datetime(PopTot['census_year'], format = '%d/%m/%Y %H:%M:%S %p')
TempPopComm = PopTot.copy()

#Drop unnecessary columns and make a new column for the year only
TempPopComm = TempPopComm.drop(['comm_code','notes', 'persons per unit', 'occupied dwellings', 'census_year'], axis = 1)
TempPopComm['Year'] =  pd.DatetimeIndex(PopTot['census_year']).year

#Rename our dataframe columns so that they can be later joined with other datasets
TempPopComm.rename(columns={'name': 'Community Name', 'population':'Population'}, inplace = True)

#Grouping by community name and year
TempPopComm.set_index(['Community Name', 'Year'], inplace = True)
TempPopComm = TempPopComm.sort_values(by = ['Community Name', 'Year'])
FinalPopComm = TempPopComm.copy()

In [12]:
display(FinalPopComm)

Population
Community Name Year            
ABBEYDALE      1968           0
               1969           0
               1970           0
               1971           0
               1972           0
...                         ...
WOODLANDS      2013        6227
               2014        6237
               2015        6229
               2016        6105
               2017        6111

[9950 rows x 1 columns]

### Crime Statistics Dataset

&nbsp;&nbsp; The Community Crime and Disorder Statistics (retrieved from OpenCalgary) categorizes the crime type and crime count according to each community. Not only does it directly provide a column in which the crime is recorded for the year, but the dataset also features a column called the 'community center point'. This column showcases the centre of the community through a pair of longitude and latitude coordinates. These values will be essential when we later plot out the crime statistics for each community across the entire city through geopandas, enabling us to visualize where crime has occurred on a geographical level.

&nbsp;&nbsp; Cleaning and filtering process is very similar, where community names and year are grouped together. Two separate dataframes are created, one to compare total crime counts per year in each community, and the other to compare the rates for each crime category. The intention of this split is to display a continuous graph that indicates crime count changes over time, as well as a separate stacked bar graph that will enable us to see which types of crime occurred the most or the least.

In [13]:
Crime = pd.read_csv("Community_Crime_and_Disorder_Statistics.csv")

In [14]:
display(Crime.head())

,Sector,Community Name,Group Category,Category,Crime Count,Resident Count,Date,Year,Month,ID,Community Center Point
0,CENTRE,ELBOYA,Crime,Street Robbery,1,1754,08/01/2019 12:00:00 AM,2019,AUG,2019-AUG-ELBOYA-Street Robbery-1,"(51.0129938009936, -114.07362044953236)"
1,CENTRE,POINT MCKAY,Crime,Residential Break & Enter,1,1343,08/01/2019 12:00:00 AM,2019,AUG,2019-AUG-POINT MCKAY-Residential Break & Enter-1,"(51.059541576145605, -114.1461459792249)"
2,SOUTH,FAIRVIEW INDUSTRIAL,Disorder,Social Disorder,22,0,08/01/2019 12:00:00 AM,2019,AUG,2019-AUG-FAIRVIEW INDUSTRIAL-Social Disorder-22,"(50.98692363059351, -114.06882221568989)"
3,NORTHWEST,COLLINGWOOD,Crime,Violence Other (Non-domestic),1,2249,08/01/2019 12:00:00 AM,2019,AUG,2019-AUG-COLLINGWOOD-Violence Other (Non-domes...,"(51.083504232297685, -114.10240136091258)"
4,CENTRE,ERLTON,Crime,Theft FROM Vehicle,8,1270,08/01/2019 12:00:00 AM,2019,AUG,2019-AUG-ERLTON-Theft FROM Vehicle-8,"(51.02632811531515, -114.06314744808363)"


#### Cleaning & Filtering: Crime By Community 2012-2019

In [15]:
#Removing unnecessary columns
CleanCrime = Crime[['Sector','Community Name', 'Crime Count','Category', 'Year']]
# display(CleanCrime)

## TOTAL CRIME PER COMMUNITY 
# reindex using Community Name to merge later with PropAssess
tempCrime1 = CleanCrime.groupby(['Community Name','Year'])
CrimeTotalCommYear = tempCrime1.sum().dropna()

## DETAILED CRIME PER COMMUNITY
# reindex using Community Name to merge later with PropAssess
tempCrime2 = CleanCrime.groupby(['Community Name','Category','Year'])
CrimeCatCommYear = tempCrime2.sum().dropna()

In [16]:
display(CrimeTotalCommYear)

Crime Count
Community Name Year             
01B            2012           12
               2013           13
               2014           12
               2015           11
               2016           10
...                          ...
WOODLANDS      2015          428
               2016          464
               2017          448
               2018          432
               2019          353

[2254 rows x 1 columns]

In [17]:
display(CrimeCatCommYear)

Crime Count
Community Name Category                      Year             
01B            Assault (Non-domestic)        2012            1
                                             2015            1
               Commercial Break & Enter      2015            1
                                             2016            1
                                             2019            2
...                                                        ...
WOODLANDS      Violence Other (Non-domestic) 2015            9
                                             2016            8
                                             2017            6
                                             2018            9
                                             2019            9

[16617 rows x 1 columns]

In [18]:
#Find the mean along the crime count columns and place it into a new column 'Average'
#Strip out any community names that have a numerical string in it (ie. 06B, 05F...)
averagecrime = CrimeTotalCommYear.unstack().copy()
averagecrime['Average'] = averagecrime['Crime Count'].mean(axis = 1)
averagecrime.reset_index(inplace = True)
averagecrime = averagecrime[~averagecrime['Community Name'].str.contains(r'[0-9]', na=False)]
averagecrime.set_index('Community Name', inplace = True)

# display(averagecrime)

In [19]:
#Find the total crime counts per community, strip out any communities with a numerical string in it.
TotCrimePerYearPerComm = CrimeTotalCommYear.unstack()['Crime Count']
TotCrimePerYearPerComm.reset_index(inplace = True)
TotCrimePerYearPerComm = TotCrimePerYearPerComm[~TotCrimePerYearPerComm['Community Name'].str.contains(r'[0-9]', na=False)] 
# TotCrimePerYearPerComm

#Find the locations (longitude and latitude) of each community center point to be later mapped
CommLoc = Crime[['Community Name', 'Community Center Point']]
CommLoc = CommLoc[~CommLoc['Community Name'].str.contains(r'[0-9]', na=False)] 
CommLoc.sort_values(by = 'Community Name', inplace = True)
CommLoc = CommLoc.drop_duplicates()
# CommLoc

#Merge both datasets on community name
combined = TotCrimePerYearPerComm.merge(CommLoc, how = 'left', on = 'Community Name')
combined = combined.drop_duplicates()
combined = combined.replace(np.nan, 0)
# combined

In [20]:
#Bring back the decimal places for the coordinates
#This part is essential to have an accurate, working map
pd.options.display.float_format = '{:.15f}'.format

#Extract the longitude and latitude for each community center point, and place them into a list that will be used to plot.
lats = []
longs = []
for i in range(0, len(combined['Community Center Point'])):
    x, y = re.findall(r'-?\d+\.?\d*', combined['Community Center Point'][i])
    lats.append(float(x))
    longs.append(float(y))

### Oil Price Dataset

&nbsp;&nbsp; The Oil Data (retrieved from Investing.com) showcases several columns relating to the physical aspects of the oil price. This includes the volume, percentage change, highest and lowest amounts. We are interested in only the date (year) and the average price. 
The oil price data were averaged over the year and limited to years 2005 to 2019. The data were formatted through datetime column to extract only the year and drop all other columns except the Price.


In [21]:
Oil_data = pd.read_csv('Oil_Data.csv')
display(Oil_data.head())

,Date,Price,Open,High,Low,Vol.,Change %
0,Sep 19,56.170000000000002,55.000000000000000,63.469999999999999,52.840000000000003,-,1.94%
1,Aug 19,55.100000000000001,57.850000000000001,57.990000000000002,50.520000000000003,14.04M,-5.94%
2,Jul 19,58.579999999999998,59.270000000000003,60.939999999999998,54.719999999999999,11.80M,0.19%
3,Jun 19,58.469999999999999,53.420000000000002,59.930000000000000,50.600000000000001,12.46M,9.29%
4,May 19,53.500000000000000,63.399999999999999,63.930000000000000,53.049999999999997,16.25M,-16.29%


#### Cleaning & Filtering: Average Oil Price 2005-2019

In [22]:
#Format so that two decimal places are acquired for the price
pd.options.display.float_format = '{:.2f}'.format

#Converting 'Date' to datetime format, so we can extract the year
# %b = 3 character month, %y = 2 digit year
Oil_data['Date'] = pd.to_datetime(Oil_data['Date'], format = '%b %y')
Oil_data['Year'] = Oil_data['Date'].dt.year

#Group by year, and find the average oil prices for all years
#Drop unwanted columns
Average_Oil_Price = Oil_data.groupby('Year', as_index = True).mean()
Average_Oil_Price = Average_Oil_Price.drop(columns = ['Open', 'High', 'Low'])

display(Average_Oil_Price.head())

,Price
Year,
2005,57.28
2006,66.96
2007,74.94
2008,98.58
2009,63.92


# Join Datasets

&nbsp;&nbsp; All datasets will be individually joined and grouped by community name and year. The joining processes for all dataframes will be consistent. In that: population, crime statistics, and average oil prices will be left joined onto the property assessment. To see the entire dataset, all three are subsequently merged together to form our complete dataframe, AllData. From there, additional refinement will be made to drop select communities that are not residential and communities with numerics. 

#### Property Assessment & Population

In [23]:
DF_Population_Assessment = PropAssessCommYear.join(FinalPopComm)

In [24]:
# display(DF_Population_Assessment)

In [25]:
#Percent change in Population & Assessed Value (for display purposes only)
#Reformat decimal place, to ensure that percentages are properly displayed
pd.options.display.float_format = '{:.4f}'.format 
DF_Percentage_Change = DF_Population_Assessment

In [26]:
# display(DF_Percentage_Change.pct_change())

#### Property Assessment & Crime Statistics

In [27]:
pd.options.display.float_format = '{:.0f}'.format
DF_Crime_Assessment = PropAssessCommYear.join(CrimeTotalCommYear)

In [28]:
# display(DF_Crime_Assessment)

#### Property Assessment & Oil Price

In [29]:
pd.options.display.float_format = '{:.2f}'.format
DF_Oil_Assessment = PropAssessCommYear.join(Average_Oil_Price)

In [30]:
#display(DF_Oil_Assessment)

#### Property Assessment & (Population, Crime Statistics, Oil Price)

In [31]:
#Joining all three datasets (Property Assessments, Crime, Oil Price, and Population)
DF_Beast = DF_Crime_Assessment.join(Average_Oil_Price)
DF_Beastv2 = DF_Beast.join(FinalPopComm)

In [32]:
#Rename dataset into AllData
AllData = DF_Beastv2.reset_index()
AllData['Community Name'] = AllData['Community Name'].replace({'SCARBORO/ SUNALTA WEST':'SCARBORO/SUNALTA WEST'})

# Remove unwanted communities, set index, and rename columns
AllData = AllData[~AllData['Community Name'].str.contains(r'[0-9]', na=False)] # removing Nameless communities
AllData = AllData[~AllData['Community Name'].str.contains('INDUSTRIAL', na=False)] # removing Nameless communities

# Communities were reviewed by population size and consistency of assessment values. Communities with missing or
# erroneous data were excluded.  Non-residential communities were also excluded.
AllData.set_index(['Community Name', 'Year'], inplace = True)
AllData.drop(index=['UNIVERSITY OF CALGARY', 'QUEENS PARK VILLAGE', 'FISH CREEK PARK', 'SOUTH FOOTHILLS',
                   'STARFIELD', 'CALGARY INTERNATIONAL AIRPORT', 'CFB - LINCOLN PARK PMQ', 'CORNERSTONE',
                   'FOOTHILLS', 'GARRISON GREEN', 'GARRISON WOODS', 'GOLDEN TRIANGLE', 'HIGHFIELD',
                   'HOTCHKISS', 'KEYSTONE HILLS', 'LIVINGSTON', 'MACEWAN', 'MCCALL', 'MERIDIAN',
                   'NORTH AIRWAYS', 'PARKHILL/STANLEY PARK', 'PINE CREEK', 'RED CARPET/MOUNTVIEW MOBILE PARK',
                   'SKYVIEW RANCH', 'SUNRIDGE', 'SYMONS VALLEY', 'UNIVERSITY DISTRICT'], level=0, inplace=True)

# replacing NaN with zeros for plotting purposes
AllData = AllData.fillna(0)
AllData['Population Change'] = AllData['Population'].pct_change()
# renaming for convenience
AllData.rename(columns={'ASSESSED_VALUE': 'Assessed Value', 'Price': 'Oil Price'}, inplace = True) 

In [33]:
display(AllData)

Assessed Value  Crime Count  Oil Price  Population  \
Community Name Year                                                       
ABBEYDALE      2005       154000.00         0.00      57.28     6040.00   
               2006       159000.00         0.00      66.96     6012.00   
               2007       228500.00         0.00      74.94     6109.00   
               2008       310000.00         0.00      98.58     6005.00   
               2009       294000.00         0.00      63.92     6032.00   
...                             ...          ...        ...         ...   
WOODLANDS      2015       467500.00       428.00      49.31     6229.00   
               2016       460000.00       464.00      44.47     6105.00   
               2017       442500.00       448.00      51.87     6111.00   
               2018       452500.00       432.00      64.54        0.00   
               2019       443500.00       353.00      57.43        0.00   

                     Population Change  
Community Name Year                     
ABBEYDALE      2005                nan  
               2006              -0.00  
               2007               0.02  
               2008              -0.02  
               2009               0.00  
...                                ...  
WOODLANDS      2015              -0.00  
               2016              -0.02  
               2017               0.00  
               2018              -1.00  
               2019                nan  

[2821 rows x 5 columns]

# Plotting

### Assessment Values Overview

In [34]:
# Json file import to get community outlines on map
census_file = './Census by Community 2019.geojson'
cendf = gpd.read_file(census_file)

# There's a geometry column which holds the geometric information as a GeoSeries
cendf['res_cnt'] = cendf['res_cnt'].astype(float).astype(int) # convert to an integer

# Let's index by community name
cendf.memory_usage(index=True, deep=True).sum() / (1024 * 1024)
#cendf = cendf.to_crs(epsg=3395) # for mercator, COMMENT THIS LINE OUT IF RUNNING WINDOWS, LEAVE LINE IN FOR MACS

# Create new smaller version for merge
newcendf = cendf[['name','geometry']].rename(columns={"name":"Community Name"})

# Merge project data with Json data
mergeprojectdata = AllData.reset_index().merge(newcendf, on='Community Name', how='left')

# Create crs for survey information and then create geopandas dataframe
crs = {'init': 'epsg:3395'}
geoprojectdata = gpd.GeoDataFrame(mergeprojectdata, crs=crs)

In [35]:
#Build an interactable drop down tab that allows user to select different years (2005-2019)
#And view median asssessed values across Calgary
@interact
def Community_Select(Year=sorted(set(geoprojectdata["Year"].tolist()))):

    #Include a geographical plot of Calgary with the median assessed values for each community
    fig, ax = plt.subplots(1, figsize=(18, 10))    
    geoprojectdata.loc[geoprojectdata['Year']==Year].plot(ax=ax, column='Assessed Value',
                                                          cmap='Blues',linewidth=0.8, edgecolor='0.8',
                                                          vmin=0, vmax=1000000) 
    #Darker hues indicate higher property assessments
    ax.axis('off')
    ax.set_title('Calgary Median Assessed Values', fontdict={'fontsize': '25', 'fontweight' : '3'})
    sm = plt.cm.ScalarMappable(cmap='Blues', norm=plt.Normalize(vmin=0, vmax=1000000))
    sm._A = []
    cbar = fig.colorbar(sm)
    plt.show()

interactive(children=(Dropdown(description='Year', options=(2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 20…

### Assessment Values and Oil Price Analysis

&nbsp;&nbsp; To visualize the impact of oil prices, we are focusing on the big picture, mainly looking at maps and scatter plots that highlight how different communities were impacted.  We first selected the years over which the oil price declined.  We chose 2015 to 2019 the range of years to consider but the plots are interactive so we can easily compare any two years.  The percentage change of median property value for the community was plotted against the distance the community is from downtown and median assessment value of the community.  We also generated a map showing the percentage change of median assessment values between years.

&nbsp;&nbsp; Analyzing the graphs and maps, we can draw a few conclusions.  First, it does appear that the median property assessments were impacted by the oil price decline.  The prices did not decrease much on average but the price growth was much smaller than both previous years in Calgary and for other Canadian cities over the years 2015 to 2019.  Second, the price change is not the same for all communities.  Both the map and the scatter plots show that the communities closer to downtown were more impacted.  Interestingly, the scatter plots show that there was more variance in the change for communities closer to downtown.  Some of both the highest and lowest percent change were for communities that are within 7 km of downtown.  Looking at these results to the map, regional trends become apparent.  The communities adjacent to downtown all exhibit a decrease in property assessment.  The majority of the distant suburbs (>10km from downtown) also generally exhibit a decrease.  However, there is a region of communities about 3-10km from downtown that exhibit a median price increase between 2015 and 2019.  We see two main reasons for this.  The first is that these communities were generally built in the 1950s to 1970s and are undergoing a period of rejuvenation, with a lot of homes undergoing substantial renovations.  Also, a lot of the communities are located in the inner-city northwest, which is home to large non-oil related employers such as the University of Calgary and Foothills and Alberta Children's Hospitals.  Could these two factors outweigh the impact of the oil price decline?

&nbsp;&nbsp; While it appears the general trend of assessment can be explained somewhat by the change in oil price, there are many other factors at play.  In addition to the factors mentioned above, several other issues such as crime and population growth can impact the median community assessments.  The rest of this project will focus on these more localized effects.


In [36]:
# Compute median for all properties in Calgary per year

CalgaryMedian = PropAssessYear[PropAssessYear["ASSESSMENT_CLASS"] == "RE"].median(axis=0)
display(CalgaryMedian)

2005        207000.00
2006        224500.00
2007        324500.00
2008        408000.00
2009        388000.00
2010        336500.00
2011        368500.00
2012        353500.00
2013        361500.00
2014        383000.00
2015        427000.00
2016        421000.00
2017        406000.00
2018        419500.00
2019        416000.00
LATITUDE        51.05
LONGITUDE     -114.08
dtype: float64

In [37]:
# Extract 30 largest downtown office towers, based on 2014 value

DTtemp = PropAssessYear[PropAssessYear["COMM_NAME"] == "DOWNTOWN COMMERCIAL CORE"]
DTtemp2 = DTtemp[DTtemp["ASSESSMENT_CLASS"] == "NR"]

# Select all downtown non-residential buildings with a 2014 value of more than $100 million
DToffices = DTtemp2[DTtemp2[2014] > 100000000].median(axis=0)

display(DToffices)

2005         57585000.00
2006         84530000.00
2007        128915000.00
2008        175305000.00
2009        172105000.00
2010        137870000.00
2011        114000000.00
2012        118435000.00
2013        171885000.00
2014        180660000.00
2015        221595000.00
2016        197205000.00
2017        156550000.00
2018        116320000.00
2019         65270000.00
LATITUDE           51.05
LONGITUDE        -114.07
dtype: float64

In [38]:

Yassets = DToffices.iloc[:15].values.tolist()

Yassets2 = CalgaryMedian.iloc[:15].values.tolist()


Yprice = []
for i in range(0,15):
    Yprice.append(Average_Oil_Price['Price'].iloc[i])

fig = make_subplots(specs=[[{"secondary_y": True}]])
x =  [2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019]
y = np.asarray(Yassets)*0.002
yy = Yprice
yyy = Yassets2
assetsname = 'Property Assessment Value'
oil = 'Oil Price'

trace1 = go.Scatter(x = x, y = y, marker = dict(line = dict(width= 1), size= 8), 
                    line = dict(color = 'red', width= 1.5), name= 'Median Price of Large Downtown Offices<br>(divided by 500 for display)')

trace2 = go.Scatter(x = x, y = yy, marker = dict(line = dict(width= 1), size= 8), 
                    line = dict(color = 'black', width= 3), name= oil)

trace3 = go.Scatter(x = x, y = yyy, marker = dict(line = dict(width= 1), size= 8), 
                    line = dict(color = 'blue', width= 1.5), name = 'Calgary Residential Median Price')



fig.add_trace(trace2, secondary_y = True);
fig.add_trace(trace1, secondary_y = False);
fig.add_trace(trace3, secondary_y = False);



fig.update_layout(title = 'Visualising Trends between Oil Price and Assessment Value', 
                    xaxis = dict(title = 'Year', showgrid = True), 
                    yaxis = dict(title = 'Property Assessment Value (CAD)', showgrid = True),
                    yaxis2 = dict(title = 'WTI Oil Price (USD)', showgrid = True))


fig.update_layout(autosize = False, width = 950,height = 400)
fig.show()





In [39]:
# (Retrieved from: https://www.movable-type.co.uk/scripts/latlong.html)
# Convert coordinate degrees to kilometers, to calculate distance of each community from downtown.
# 1 degree north = 111.2km
# 1 degree west = 69.98km
npnorth = np.asarray(lats)*111.2
npwest = np.asarray(longs)*69.98
dtnorth = 51.04727001996*111.2
dtwest = -114.0695922107*69.98
distDT = ((npnorth - dtnorth)**2 + (npwest - dtwest)**2)**0.5

#Create a new dataframe with the column 'distDT' to show the community's distance from downtown in kilometers
disttoDT = combined[['Community Name', 'Community Center Point']]
disttoDT['distDT'] = distDT

#Replace the spacing between the community names of Scarboro and Sunalta West
disttoDT['Community Name'] = disttoDT['Community Name'].replace({'SCARBORO/ SUNALTA WEST':'SCARBORO/SUNALTA WEST'})
disttoDTnodup = disttoDT.drop_duplicates('Community Name').set_index("Community Name")

#Reinsert distance and coordinates into the dataframe to be displayed
AllDataDT =  AllData.merge(disttoDTnodup, left_index=True, right_index=True, how='left').reset_index()

In [40]:
#Compute and graph ratios of assessed value from 2015 to 2018
#Build an interactable that allows the user to select two separate years
#Allow the user to also select between two types of graph: Distance to downtown or Assessment Value over Percent Change
@interact
def Community_Select(Year=sorted(set(geoprojectdata["Year"].tolist())),
                     Year2=sorted(set(geoprojectdata["Year"].tolist())),
                    Graph_Type=['Distance to Downtown vs Percent Change', 'Assessment Value vs Percent Change']):
    
    #Give the user a prompt to select a year before displaying the chart
    if Year >= Year2:
        print('Please select Year2 to be larger than Year')
    else:   
        tempforratio = AllDataDT.loc[AllDataDT['Year'].isin([Year,Year2])]
        tempforratio['percentchange'] = tempforratio['Assessed Value'].pct_change()*100

        
        if Graph_Type == 'Distance to Downtown vs Percent Change':
            fig = px.scatter(tempforratio[tempforratio['Year'] == Year2], x="distDT", y="percentchange", 
                             title='Graph of Distance to Downtown vs Percentage Change in Assessed Value<br>'
                             + str(Year) + ' to ' + str(Year2), size='Assessed Value',
                             size_max = 30, hover_name='Community Name')
        #    fig.update_yaxes(range=[-100, 100])

            fig.update_layout(xaxis = dict(title = 'Distance from Downtown (km)', showgrid = True), 
                                yaxis = dict(title = 'Percentage Change in property Assessment', showgrid = True))
            fig.show()
        
        else:
            fig = px.scatter(tempforratio[tempforratio['Year'] == Year2], x="Assessed Value", y="percentchange", 
                             title='Graph of Assessment Value vs Percent Change in Assessed Value<br>'
                             + str(Year) + ' to ' + str(Year2), size='distDT',
                             size_max = 30, hover_name='Community Name')
        #    fig.update_yaxes(range=[-100, 100])

            fig.update_layout(xaxis = dict(title = 'Community Median Property Assessment (CAD)', showgrid = True), 
                                yaxis = dict(title = 'Percentage Change in property Assessment', showgrid = True))
            fig.show()

interactive(children=(Dropdown(description='Year', options=(2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 20…

In [41]:
@interact
def Community_Select(Year=sorted(set(geoprojectdata["Year"].tolist())),
                     Year2=sorted(set(geoprojectdata["Year"].tolist()))):

    if Year >= Year2:
        print('Please select Year2 to be larger than Year')
    else:  
        tempforratiomap = geoprojectdata.loc[geoprojectdata['Year'].isin([Year,Year2])]
        tempforratiomap['percentchange'] = tempforratiomap['Assessed Value'].pct_change()*100

        fig, ax = plt.subplots(1, figsize=(18, 10))    
        tempforratiomap.loc[tempforratiomap['Year']==Year2].plot(ax=ax, column='percentchange', cmap='seismic',linewidth=0.8, edgecolor='0.8', vmin=-30, vmax=30) 
        ax.axis('off')
        ax.set_title('Calgary Median House Prices', fontdict={'fontsize': '25', 'fontweight' : '3'})
        sm = plt.cm.ScalarMappable(cmap='seismic', norm=plt.Normalize(vmin=-30, vmax=30))
        sm._A = []
        cbar = fig.colorbar(sm)
        plt.show()

interactive(children=(Dropdown(description='Year', options=(2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 20…

### Assessment Values and Population Change Analysis

*Cross - Community Comparison 1: Bowness and Evanston*

&nbsp;&nbsp; The community of Bowness was chosen for comparison as it had one of the most stable population changes across the entire city. Since it was fully built out early on, there are no major observed increases or decreases in its population count. On the other hand, Evanston was picked since it showed one of the highest overall population growth rates across the entire city. Altogether, these two become great candidates to conduct cross-community comparisons on the relationship between percent population change and median property assessments.

&nbsp;&nbsp; Beginning with the graph on Bowness, we see green line chart overlaying the bar chart that displays the percent population change between each year. There is no value for 2005, because this is the first reference point to calculate percentage change. Overall, the aim of this graph is to show how mild the fluctuations are for population change (range is between -3.24% and +4.85%). Comparatively, Evanston shows a drastic population growth rate in the early years between 2005 and 2008, reaching a peak of 59.15% increase and maintaining a consistently strong growth throughout the rest of the years.
Looking into the assessment values (bar chart), there is a large jump in the values of the assessment between 2005 and 2008. This change is largely attributed to the spike in oil prices which occurred during this timeframe, as it can be seen in the median property assessments across every community. As a result, it should be ignored when considering the effects of population change on the assessed values for all communities.

&nbsp;&nbsp; When viewing the two communities together, we see there is a very similar trend in median property assessments that appears regardless of the population growth rate.  Both groups show a drop in assessed values after 2008, and a steady increase thereafter. This is a strong indicator that population growth rate does not play as strong of a role as we had initially thought. As a result, we cannot conclude that there is an effect of population growth rate on a community’s median assessed value for this case. To show that this is not a random outcome, we looked at another pair of communities with similar features (stable population count vs higher population growth)
<br><br>

*Cross – Community Comparison 2: Abbeydale & Tuscany*

&nbsp;&nbsp; Abbeydale and Tuscany are chosen for our next analysis. Abbeydale was chosen primarily for its stable population rate, fluctuating between a max of +3.94% and a minimum of -4.13%. Comparatively, Tuscany shows a larger population growth rate across all the years, starting with a maximum of +19.11% and gradually stabilizing to a minimum of about -0.3%. While Tuscany does not show as strong of a growth rate compared to Evanston or others developed more recently, we chose to use this community over others because it had access to the property assessments for all the years between 2005 and 2017. This helps to keep the analysis more consistent. 
Looking into the assessed values for both charts, we see a similar increase between 2005 to 2008 and a drop in the following year. Both charts also exhibit a steady increase in median property assessments thereafter. One prominent difference is the scaling of the median assessed values. However, this cannot be attributed to the change in population growth rates as developing communities generally have more modern housing that improves its physical characteristics (such as size, land, aesthetics, etc). Since this trend in property assessments appears in both communities regardless of population change, we cannot conclude that population growth rate influences property assessments in Calgary. 
<br><br>

*Cross - Community Comparison Results*

&nbsp;&nbsp; Overall, both pairs of community comparisons do not show a strong relationship between population change and property assessments in Calgary. There is a similar trend in the median property assessments that appears regardless of the population growth rate for all the graphs we looked at. Interestingly, this contradicts what was concluded in previous studies found by Buchanan & Weber (1980 & 1982). One potential reason for this could be that Calgary is a city that suffers from urban sprawl, whereas Oregon (the city that the study was focused on) does not to the same extent. The lack of dense housing in our communities may impact the measure of the median property assessment, as there are less residential properties to explore. More dense communities could show a different representation of community property assessment. 


In [42]:
#Create an interactive pulldown bar that lets you choose which community you want to view:
@interact
def Community_select(Community=sorted(AllData.index.get_level_values(0).unique().tolist())):
    #Plot both a bar chart and a line chart together
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    #Plot the x values as the years in which all the data appeared in
    #Plot the yvalues for the bar chart as median assessed values for that community
    #Plot the y values for the line chart as changes in percent population
    x_assessment = list(AllData['Population Change'].unstack().columns[:-2])
    y_assessment = AllData['Assessed Value'].unstack().loc[Community]
    y_population = AllData['Population Change'].unstack().loc[Community]

    #Add traces for the information to be graphed
    trace1 = go.Bar(x = x_assessment, y = y_assessment, name = 'Assessment Values', marker = dict(color = 'palevioletred'))
    trace2 = go.Scatter(x = x_assessment, y = y_population, marker = dict(line = dict(width= 1)), 
                        line = dict(color = 'lightgreen', width= 1.5), name = 'Percent Population Change')

    #Add two y-axis, one for assessment value, the other for percent population change
    fig.add_trace(trace1, secondary_y = False);
    fig.add_trace(trace2, secondary_y = True);

    #Configure display
    fig.update_layout(title = 'Assessment Value and Percent Population Change in %s between 2005-2017' %Community, 
                      xaxis = dict(title = 'Year',showgrid = True, nticks = 15), 
                      yaxis = dict(title = 'Assessment Value in CAD', showgrid = True),
                      yaxis2 = dict(title = 'Percent Population Change', showgrid = True))
    
    #Update layout
    fig.update_layout(autosize = False, width = 950,height = 600)
    fig['layout']['yaxis2'].update(range = [-0.2, 0.6])
    fig.show()

interactive(children=(Dropdown(description='Community', options=('ABBEYDALE', 'ACADIA', 'ALBERT PARK/RADISSON …

### Assessment Values and Crime Count Analysis

&nbsp;&nbsp; Although it is difficult to directly attribute declining property assessments to criminal incidents, it is easy to think of how these two factors are related. New residents will tend to avoid neighborhoods with elevated crime rates, while current residents might look to move elsewhere. On the other hand, neighborhoods that are considered safe might be more expensive due to higher demand. Visualising the data on property assessment values and crime rates in select communities in Calgary will further shed some light on whether or not these two things are related.
<br><br>

*Community: Beltline (highest crime counts)*

&nbsp;&nbsp; This community was of particular interest because it has the highest average crime counts in Calgary from 2012 till 2019.
As can be seen from the graph, crime counts in this community rose from ~6600 in 2012 to ~9000 in 2015. During that time however, the property assessment value for the community rose from 190k to around 227k. After that 13% bump in 2015, property assessments in that community started to decline steadily, reaching 185.5k in 2019, with crime counts showing a dip in 2016 to ~8300, only to go back up to almost exactly the same as the 2015 count. 
The purpose of this analysis is to determine whether or not crime counts in a community have an effect on property assessment values in that community. 

&nbsp;&nbsp; Assuming that a decrease in assessment values will lag a little behind the increase in crime counts, we would expect the values to start decreasing after 2014, however in 2015 this community saw a 13% increase in property assessment values. Interestingly, after 2015 we see that the assessment values started to steadily decrease as the crime counts rose. 
At first sight, it would seem possible to establish a hypothesis that after an increase in crime counts from 2012 to 2015, property assessment values started to decrease as a result. But it is difficult to assert that this is the case in this community in particular due to its relationship with the oil and gas industry. From 2014 to 2015, the oil price decreased dramatically from 91.23USD/bbl to 49.3USD/bbl, followed by the decrease in property assessment values. So, it is tough to accurately state which one, or both, of these parameters lead to the decrease in assessment values. 

To avoid such dependencies, other communities were chosen that were less impacted by the decrease in oil price. 
<br><br>


*Community: Forest Lawn (second highest crime count)*

&nbsp;&nbsp; This community is ranked third in terms of average crime counts from 2012 to 2019, with an average crime count of 2612 crimes per year. If our hypothesis is correct, we would expect to see a decrease in property assessment values lagging a year or two behind an increase in crime counts. Interestingly we see quite the opposite in this community. From 2012 till 2014, crime counts averaged around 2500, while assessment values increased slightly from 248k to 268k. Both these numbers saw a bump in 2015, where crime counts rose 13% to around 2800 and assessment values rose 16% to 311k. Following that, up until 2018, assessment values remained somewhat constant, with crime counts increasing to around 3000. 
It is also apparent that the decrease in oil price did not affect this community.


&nbsp;&nbsp; Going by the trends shown by the graph, we can conclude that our initial hypothesis is incorrect. An increase in crime counts does not lead to a decrease in property assessment values. However, we must also be careful not to conclude the opposite, that an increase in crime counts lead to an increase in property assessment values. As any data scientist, or indeed any person in general must know, correlation does not imply causation. 
<br><br>

*Community: Bel-Aire (lowest crime count)*

&nbsp;&nbsp; This community was chosen because it has the lowest average crime count from 2012 to 2019 at 22 crimes per year. 
For this community, it seems at first glance that our hypothesis holds. A decrease in crime counts from 2013 to 2014 is followed by an increase in property assessment values in 2015. Then a drastic increase of crime counts from just 4 to 35 from 2014 to 2016 is followed by a decrease in assessments values from 1.8M to 1.64M. However, this community has too few people and crimes to be considered an adequate proof for our hypothesis.
<br><br>

*Community: Sage Hill (Highest assessment)*

&nbsp;&nbsp; Sage Hill has the highest average property assessment value from 2012 till 2019.  It shows significant increases particularly in 2014 and 2018. However, crime counts were consistently increasing from 89 in 2014 to 315 in 2018. This community does not conform to our hypothesis that an increase in crime counts would negatively impact property assessments, even when we account for the increase in population. Crime per person increased, but assessments values did not decrease. 
<br><br>

*Community: Chinatown (Lowest assessment)*

&nbsp;&nbsp; Chinatown shows the lowest, and probably most consistent property assessment values since 2012 at around 100k. As crime counts rose from 161 to 247, there was no significant change in property assessment values in this community, forcing us to reject our hypothesis in this case.  
<br>

*Community: Panorama Hills (Highest Population)*

&nbsp;&nbsp; This community was chosen because it has the highest average population in Calgary from 2012 to 2019 at 24,795 residents. It must be noted that this community does not have the highest crime count, nor the lowest crime/person value. 
Again, looking at the graph makes us lean towards accepting out hypothesis as true. A decrease in crime counts from 668 in 2012 to 541 in 2014 was accompanied by a small increase in assessment values, followed by a 10% increase in 2015. However, an increase in crime counts to 740 in 2015 only a small decrease in assessment values from 496.5k in 2016 to 474.5k in 2017.


&nbsp;&nbsp; After looking at trends of property assessments and crime counts for a few more random communities, it can be concluded that there does not appear to be strong consistent evidence in support of the hypothesis that crime counts and property assessment values are related, either positively or inversely. For us to make such a claim, further analysis should be conducted to form a probabilistic model that looks at the possible correlation between these two factors. 


#### Bar graph for Detailed Crime Counts in selected Community

In [43]:
@interact
def Community_Select(Community=sorted(set(AllData.index.get_level_values(0).tolist()))):

    newdf = pd.DataFrame(CrimeCatCommYear.loc[Community])
    newdf = newdf.unstack(level = 0)
    newdf = newdf['Crime Count']

    traces = []
    for i in range(1, len(newdf.columns)):
        traces.append(go.Bar(x = np.sort(CleanCrime['Year'].unique()), y = newdf[newdf.columns[i]], name = newdf.columns[i]))

    data = traces

    layout = go.Layout(title = 'Distribution of Crime based on Category in %s 2012-2019' %Community, 
                       barmode = 'stack',
                       xaxis_title = 'Year',
                       yaxis_title = 'Crime Count',
                       autosize = False,
                       height = 700,
                       width = 1000)

    fig = go.Figure(data = data, layout = layout)
    fig.show()

interactive(children=(Dropdown(description='Community', options=('ABBEYDALE', 'ACADIA', 'ALBERT PARK/RADISSON …

#### Assessment Values and Crime

In [44]:
@interact
def Community_Select(Community=sorted(set(AllData.index.get_level_values(0).tolist()))):
    
    Ycrimes = AllData.unstack()['Crime Count'].loc[Community][7:15].tolist()
    Yassets = AllData.unstack()['Assessed Value'].loc[Community][7:15].tolist()

    fig = make_subplots(specs=[[{"secondary_y": True}]])
    x =  np.sort(CleanCrime['Year'].unique())

    trace1 = go.Bar(x = x, y = Yassets, name = 'Assessment Value' , marker = dict(color = 'palevioletred'))

    trace2 = go.Scatter(x = x, y = Ycrimes, marker = dict(line = dict(width= 1), size= 8), 
                        line = dict(color = 'lightsalmon', width= 1.5), name = 'Crime Count')

    fig.add_trace(trace1, secondary_y = False);

    fig.add_trace(trace2, secondary_y = True);

    fig.update_layout(title = 'Crime and Assessment Values in %s for 2012-2019' %Community,
                      xaxis = dict(title = 'Year',showgrid = True), 
                      yaxis = dict(title = 'Assessment Value in CAD', showgrid = True),
                      yaxis2 = dict(title = 'Crime Count', showgrid = True), 
                      autosize = False, width = 950,height = 600)
    fig.show()

interactive(children=(Dropdown(description='Community', options=('ABBEYDALE', 'ACADIA', 'ALBERT PARK/RADISSON …

#### Distribution of Crime Occurrences

In [45]:
# Choose Year to view stats
Year = 2019

In [46]:
# color of scatter points based on crime count
crimetots = list(combined[Year])
crimetext = []
for i in range(0,len(crimetots)):
    crimetext.append(str(int(crimetots[i])))
crimetext
hovertextcrime = ['Crime Count: ' + i for i in crimetext]

commtext = []
for i in range(0,len(combined['Community Name'])):
    commtext.append((combined['Community Name'][i]))
commtext
hovertextcomm = ['Community: ' + i for i in commtext]

hovertext = [m + ', ' + n for m,n in zip(hovertextcomm, hovertextcrime)]

In [47]:
try:
    file = open('mapbox.token', 'r')
    for line in file:
        token = line.rstrip()
    file.close()
except FileNotFoundError as fnfe:
    print(fnfe)

data = [
    go.Scattermapbox(
        lat=lats,
        lon=longs,
        mode='markers',
        marker=dict(color = crimetots, cmin=100, cmax = 1500,
            size=15, colorscale = 'amp',showscale = True, reversescale = False
        ),
        text=hovertext, hoverinfo='text'
    )
]

layout = go.Layout(
    title='Crime Occurrences',
    autosize=True,
    height=1024,
    hovermode='closest',
    mapbox=dict(
        style="outdoors", #streets, light, dark, satellite
        accesstoken=token,
        bearing=0,
        center=dict(
            lat=51.0486,
            lon=-114.0708
        ),
        pitch=0,
        zoom=10,
    ),
)

fig = dict(data=data, layout=layout)
py.iplot(fig)

## Visualising Trends

#### Interactive plot of Assessment Values, Population, Crime Counts, and Oil Price

In [48]:
@interact
def Community_Select(Community=sorted(set(AllData.index.get_level_values(0).tolist()))):
    
    Ycrimes = AllData.unstack()['Crime Count'].loc[Community][7:15].tolist()
    Yassets = AllData.unstack()['Assessed Value'].loc[Community][7:15].tolist()
    Ypop = AllData.unstack()['Population'].loc[Community][7:13].tolist()
    y4 = []
    for i in range(0,len(Ycrimes)-2):
        y4.append(Ycrimes[i]/Ypop[i])

    fig = make_subplots(specs=[[{"secondary_y": True}]])
    x =  np.sort(CleanCrime['Year'].unique())

    assetsname = 'Property Assessment Value'
    popname = 'Population'
    pricename = 'Oil Price'
    crimename = 'Crime Count'
    crimeperpop = 'Crime per Person'

    trace1 = go.Bar(x = x, y = Yassets, name = assetsname, marker = dict(color = 'palevioletred'))

    trace2 = go.Scatter(x = x, y = Ycrimes, marker = dict(line = dict(width= 1), size= 8), 
                        line = dict(color = 'lightsalmon', width= 1.5), name= crimename)

    trace3 = go.Scatter(x = x, y = Ypop, marker = dict(line = dict(width= 1), size= 8), 
                        line = dict(color = 'royalblue', width= 1.5), name= popname)

    trace4 = go.Scatter(x = x, y = Yprice, marker = dict(line = dict(width= 1), size= 8), 
                        line = dict(color = 'brown', width= 1.5), name= pricename)

    trace5 = go.Scatter(x = x, y = y4, marker = dict(line = dict(width= 1), size= 8), 
                        line = dict(color = 'green', width= 1.5), name= crimeperpop)

    fig.add_trace(trace1, secondary_y = False);
    fig.add_trace(trace2, secondary_y = True);
    fig.add_trace(trace3, secondary_y = True);
    fig.add_trace(trace4, secondary_y = True);
    fig.add_trace(trace5, secondary_y = True);

    fig.update_layout(title = 'Visualising Trends for %s' %Community,
                      xaxis = dict(title = 'Year', showgrid = True), 
                      yaxis = dict(title = 'Property Assessment Value in CAD', showgrid = True),
                      yaxis2 = dict(title = 'Crime Count / Population / Oil Price', showgrid = True),
                      autosize = False, width = 950,height = 600)
    
    fig.show()

interactive(children=(Dropdown(description='Community', options=('ABBEYDALE', 'ACADIA', 'ALBERT PARK/RADISSON …

## Conclusion 

&nbsp; &nbsp; This data exploration was an attempt to look into the factors that might affect the property assessment values in the city of Calgary. Trends of assessment values were compared against trends of oil price, population change, and crime counts on a community basis. 
Our visual analysis concluded that there indeed might be a correlation between assessment values and the oil price, but did not provide adequate evidence to allow us to conclude the same for population change. As for crime counts, some communities exhibited a correlation with assessment values, and some did not. Therefore we cannot be certain as to the strength of the relationship between these two factors.
Whatever the case may be, if we want to be more certain of the results of our visual analysis, a more rigorous statistical investigation must be conducted to lend credence to our claims. 

## References

Buchanan, S.C. & Weber, B.A. (1982) ‘Growth and Residential Property Taxes: A Model for Estimating Direct and Indirect Population Impacts’, The University of Wisconsin Press, 58(3), p324-337.<br><br>
Calgary Open Data. (2018) Calgary’s Population 1958 - 2019. Available from:
https://data.calgary.ca/Demographics/Civic-Census-Results-1958-2019/rmai-qvzh [Accessed
27th September 2019] <br><br>
Calgary Open Data. (2018) Property Assessments. Available from:
https://data.calgary.ca/dataset/Property-Assessments/6zp6-pxei [Accessed 27th September
2019]<br><br>
Calgary Open Data. (2018) Community Crime and Disorder Statistics. Available from:
https://data.calgary.ca/Health-and-Safety/Community-Crime-and-Disorder-Statistics/848s-4m4z
[Accessed 27th September 2019]<br><br>
Campbell, Douglas A. “Crime and Property Values.” The Free Library, University of Memphis,
2007, retrieved September 27, 2019 from
https://www.thefreelibrary.com/Crime%20and%20property%20values.-a0170114047<br><br>
Glink, I. (2013, May 20). Can a rise in crime increase your property value? Retrieved September
27, 2019, from
https://www.cbsnews.com/news/can-a-rise-in-crime-increase-your-property-value/<br><br>
Investing.com. (2019) Crude Oil WTI Futures Historical Data. Available from:
https://ca.investing.com/commodities/crude-oil-historical-data [Accessed 28th September 2019]<br><br>
Maximino, M. (2017, February 16). The impact of crime on property values. Retrieved
September 27, 2019, from
https://journalistsresource.org/studies/economics/real-estate/the-impact-of-crime-on-property-val
ues-research-roundup/<br><br>
Property Assessments 2019, City of Calgary, viewed September 30, 2019,
<https://www.calgary.ca/PDA/Assessment/Pages/Property-Assessment.aspx>.<br><br>
Weber, B.A. & Buchanan, S.C. (1980) ‘The Impact of Population Growth on Residential Property Taxes, Western Journal of Agricultural Economics, 5(2), p177-184.